# Show used config values

In [ ]:
from collections import defaultdict
import os
import json
from helpers import RESULTS_DIR

# Show possible values for the config files
config_values = defaultdict(set)
folders = [f for f in os.listdir(RESULTS_DIR) if os.path.isdir(os.path.join(RESULTS_DIR, f))]
for folder in folders:
    with open(os.path.join(RESULTS_DIR, folder, 'config.json'), 'r') as f:
        config = json.load(f)
    for key, value in config.items():
        config_values[key].add(str(value))

# Drop experiment name, best, mean, std
ignore = ['experiment_name']
for fitness_method in ['default', 'balanced', 'defensive']:
    ignore.extend([f'best_{fitness_method}', f'mean_{fitness_method}', f'std_{fitness_method}', f'Q5_{fitness_method}', f'Q95_{fitness_method}'])
for key in ignore:
    config_values.pop(key, None)
display(config_values)

# Count runs per config

In [ ]:
from collections import defaultdict
import os
import json
import shutil
from helpers import RESULTS_DIR

ignore = ['experiment_name']
for fitness_method in ['default', 'balanced', 'defensive']:
    ignore.extend([f'best_{fitness_method}', f'mean_{fitness_method}', f'std_{fitness_method}', f'Q5_{fitness_method}', f'Q95_{fitness_method}'])

# Count number of runs per unique config
runs_per_config = defaultdict(int)
folders = [f for f in os.listdir(RESULTS_DIR) if os.path.isdir(os.path.join(RESULTS_DIR, f))]
for folder in folders:
    with open(os.path.join(RESULTS_DIR, folder, 'config.json'), 'r') as f:
        config = json.load(f)
    for key in ignore:
        config.pop(key, None)
    if config['gen'] < config['gens']-1:
        print(f'Incomplete run: {folder}. {config["gen"]} out of {config["gens"]} generations.')
        # Delete folder and contents
        shutil.rmtree(os.path.join(RESULTS_DIR, folder))
        continue
    runs_per_config[str(config)] += 1

# Show number of runs per unique config
for config, runs in sorted(runs_per_config.items(), key=lambda x: x[0], reverse=True):
    print(f'{runs} runs for config {config}')

# File to show plots for the evolution runs

In [ ]:
from plotting import create_plot
from helpers import find_folders, RESULTS_DIR

default_config = { # First experiment with 10 runs
    "randomini": "no",
    "multi_ini": False,
    "n_hidden_neurons": 10,
    "pop_size": 100,
    "mutation_rate": 0.2,
    "normalization_method": "default",
    "fitness_method": "default",
    "pick_parent_method": "tournament",
    "survivor_method": "multinomial",
    "crossover_method": "none",
    "mutation_type": "normal",
    "domain_upper": 1,
    "domain_lower": -1,
}

default_config = { # Second experiment with 30 runs
    "randomini": "no",
    "multi_ini": False,
    "n_hidden_neurons": 10,
    "pop_size": 100,
    "mutation_rate": 0.2,
    "normalization_method": "default",
    "fitness_method": "rank",
    "pick_parent_method": "multinomial",
    "survivor_method": "multinomial",
    "crossover_method": "none",
    "mutation_type": "normal",
    "domain_upper": 1,
    "domain_lower": -1,
}

# variable = {"fitness_method": ["default", "balanced"]}  	        # seems pretty much the same
# variable = {"fitness_method": ["default", "rank"]}  	            # pretty much the same
# variable = {"normalization_method": ["default", "domain_specific"]} # seems consistently slightly worse (puzzling)
# variable = {"normalization_method": ["default", "around_0"]}        # mean is similar, best is sometimes better, sometimes worse, 
# variable = {"pick_parent_method": ["tournament", "greedy"]}         # way better mean obviously, inconsistent best
# variable = {"pick_parent_method": ["tournament", "multinomial"]}    # multinomial seems consistently better
# variable = {"survivor_method": ["multinomial", "greedy"]}           # way better mean obviously, inconsistent best
# variable = {"survivor_method": ["multinomial", "tournament"]}           # tournament seems consistently worse
# variable = {"crossover_method": ["none", "default"]}                # consistently worse mean, similar best
# variable = {"crossover_method": ["none", "ensemble"]}               # consistently worse mean, inconsistent best
# variable = {"randomini": ["no", "yes"]}                           # just luck based
# variable = {"multi_ini": [False, True]}                           # obviously worse scores but better for multi ini eval
variable = {"mutation_type": ["normal", "stochastic_decaying"]}   # better mean, maybe slightly better best

for enemies in [[1], [2], [3], [4], [5], [6], [7], [8]]:
    print(f"Enemies: {enemies}")
    config = default_config.copy()
    config.update({"enemies": enemies})

    folders_list = []
    for value in list(variable.values())[0]:
        config.update({list(variable.keys())[0]: value})

        folders_list.append(find_folders(config))

    create_plot(variable, *folders_list, figsize=(5,3), results_dir=RESULTS_DIR, fitness_method='default')


# Boxplots

number of wins/defeats isn't great as a metric for a boxplot because it only ranges between 0 and 8

In [ ]:
# Boxplots
from plotting import create_boxplot
from helpers import find_folders, RESULTS_DIR

default_config = {
    "randomini": "no",
    "multi_ini": False,
    "n_hidden_neurons": 10,
    "pop_size": 100,
    "mutation_rate": 0.2,
    "normalization_method": "default",
    "fitness_method": "default",
    "pick_parent_method": "tournament",
    "survivor_method": "multinomial",
    "crossover_method": "none",
    "mutation_type": "normal",
    "domain_upper": 1,
    "domain_lower": -1,
}
default_config = {
    "randomini": "no",
    "multi_ini": False,
    "n_hidden_neurons": 10,
    "pop_size": 100,
    "mutation_rate": 0.2,
    "normalization_method": "default",
    "fitness_method": "rank",
    "pick_parent_method": "multinomial",
    "survivor_method": "multinomial",
    "crossover_method": "none",
    "mutation_type": "normal",
    "domain_upper": 1,
    "domain_lower": -1,
}
# variable = {"fitness_method": ["default", "balanced"]} # not much difference
# variable = {"normalization_method": ["default", "domain_specific"]} # not much difference
# variable = {"pick_parent_method": ["tournament", "multinomial"]}
# variable = {"multi_ini": [False, True]} # obviously much better scores for multi ini eval
variable = {"mutation_type": ["normal", "stochastic_decaying"]} # sometimes slightly better, sometimes slightly worse
# variable = {"crossover_method": ["none", "ensemble"]} # sometimes slightly better, sometimes slightly worse

# Same as above but with boxplots
for enemies in [[1], [2], [3], [4], [5], [6], [7], [8]]:
    print(f"Enemies: {enemies}")
    config = default_config.copy()
    config.update({"enemies": enemies})

    folders_list = []
    for value in list(variable.values())[0]:
        config.update({list(variable.keys())[0]: value})
    
        folders_list.append(find_folders(config))

    create_boxplot(variable, *folders_list, figsize=(5,3), results_dir=RESULTS_DIR, randomini_eval=False, multi_ini_eval=False)

# Hardest enemies

In [ ]:
# Make boxplot for each enemy based on the best fitness in each run
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
from helpers import find_folders, RESULTS_DIR

def create_enemy_boxplot(config={}, randomini_eval=False, multi_ini_eval=False, figsize=(5,3), results_dir=RESULTS_DIR, save_png=False, metric='gain'):

    add_str = ""
    if randomini_eval:
        add_str = "_randomini"
    elif multi_ini_eval:
        add_str = "_multi-ini"

    data = []
    shortest_folders_len = 1e6
    for enemy in range(1,9):
        config.update({"enemies": [enemy]})
        folders = find_folders(config)
        shortest_folders_len = min(shortest_folders_len, len(folders))

        if not folders:
            print(f'No folders found for enemy: {enemy}')
            return


        runs = []
        for folder in folders:
            # Create empty df with columns for [gain, fitness, fitness_balanced, n_wins]
            df = pd.DataFrame(columns=['gain', 'fitness', 'fitness_balanced', 'wins'])

            # Read results from eval_best.json
            with open(f'{results_dir}/{folder}/eval_best{add_str}.json', 'r') as f:
                saved = json.load(f)
            df = pd.DataFrame(saved["results"])
            # Turn wins list into number of wins if wins is a list type
            if type(df['wins'][0]) == list:
                df['wins'] = df['wins'].apply(lambda x: sum(x))

            # Average over the 5 evals, keep dims
            df = df.mean(axis=0)
            df = df.to_frame().transpose()
            runs.append(df)
        runs = pd.concat(runs, axis=0)

        data.append(runs[metric])
    print(f'Shortest folder list: {shortest_folders_len}')
    plt.figure(figsize=figsize)

    # Plot boxplot(s)
    plt.boxplot(data)
    plt.xticks(range(1,9), range(1,9))
    plt.title(f'{metric.capitalize()} boxplot')

    plt.xlabel('Enemy')
    plt.ylabel(metric.capitalize())
    if save_png:
        if not os.path.exists(f'plots/{str(variable)}'):
            os.makedirs(f'plots/{str(variable)}')
        plt.savefig(f'plots/{str(variable)}/{metric}_boxplot.png')
    plt.show()

print('Evaluated on all initial enemy positions')
create_enemy_boxplot(config={
        "randomini": "no",
        "multi_ini": False,
    },
    randomini_eval=False, multi_ini_eval=True, figsize=(5,3), results_dir=RESULTS_DIR, save_png=False, metric='gain')

print('Evaluated on only default enemy position')
create_enemy_boxplot(config={
        "randomini": "no",
        "multi_ini": False,
    },
    randomini_eval=False, multi_ini_eval=False, figsize=(5,3), results_dir=RESULTS_DIR, save_png=False, metric='gain')

# Delete folders without config

In [ ]:
import os
import json
from helpers import RESULTS_DIR

# Get all folders
folders = [f for f in os.listdir(RESULTS_DIR) if os.path.isdir(os.path.join(RESULTS_DIR, f))]
for folder in folders:
    # Check if config.json exists
    if not os.path.exists(os.path.join(RESULTS_DIR, folder, 'config.json')):
        print(f'No config.json in folder: {folder}, deleting folder')
        for file in os.listdir(os.path.join(RESULTS_DIR, folder)):
            os.remove(os.path.join(RESULTS_DIR, folder, file))
        os.rmdir(os.path.join(RESULTS_DIR, folder))
        continue
    # else:
    #     with open(os.path.join(RESULTS_DIR, folder, 'config.json'), 'r') as f:
    #         config = json.load(f)
    #     if "best_log" not in config:
    #         print(f'No best_log in config.json in folder: {folder}, deleting folder')
    #         # Delete folder and contents
    #         for file in os.listdir(os.path.join(RESULTS_DIR, folder)):
    #             os.remove(os.path.join(RESULTS_DIR, folder, file))
    #         os.rmdir(os.path.join(RESULTS_DIR, folder))
    #         continue

# Update all configs

In [ ]:
import os
import json
from helpers import RESULTS_DIR

# Add variable to all config.json files
variable = "mutation_type"
default_value = "normal"

folders = [f for f in os.listdir(RESULTS_DIR) if os.path.isdir(os.path.join(RESULTS_DIR, f))]
for folder in folders:
    with open(os.path.join(RESULTS_DIR, folder, 'config.json'), 'r') as f:
        config = json.load(f)
    if variable not in config:
        config.update({variable: default_value})
    with open(os.path.join(RESULTS_DIR, folder, 'config.json'), 'w') as f:
        json.dump(config, f, indent=4)

# Multi Enemy

In [ ]:
# Make matplotlib plots interactive
%matplotlib widget

In [ ]:
from plotting import create_plot
from helpers import find_folders, RESULTS_DIR

default_config = { # Second experiment with 30 runs
    "randomini": "no",
    "multi_ini": False,
    "n_hidden_neurons": 10,
    "pop_size": 100,
    "mutation_rate": 0.2,
    "normalization_method": "default",
    "fitness_method": "rank",
    "pick_parent_method": "multinomial",
    "survivor_method": "multinomial",
    "crossover_method": "none",
    "mutation_type": "stochastic_decaying",
    "domain_upper": 1,
    "domain_lower": -1,
}


variable = {"mutation_type": ["normal", "stochastic_decaying"]}   # better mean, maybe slightly better best

for enemies in [[1,2,3,4,5,6,7,8],[3,4,6,7]]:
    print(f"Enemies: {enemies}")
    config = default_config.copy()
    config.update({"enemies": enemies})

    folders_list = []
    for value in list(variable.values())[0]:
        config.update({list(variable.keys())[0]: value})

        folders_list.append(find_folders(config))

    create_plot(variable, *folders_list, figsize=(5,3), results_dir=RESULTS_DIR, fitness_method='default', plot_separate_lines=True)
